This notebook fits a Convolutional Neural Network inorder to detect Hindi characters.

    Let's import the necessary modules:

In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt


#set up the test, train directories:
train_dir = "datasets/DevanagariHandwrittenCharacterDataset/Train"
test_dir = "datasets/DevanagariHandwrittenCharacterDataset/Test"

Let's try data augmentation:


In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

    Generating the data for testing and training:

In [3]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),
    batch_size=128,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(32, 32),
    batch_size=128,
    class_mode='categorical'
)


Found 78200 images belonging to 46 classes.
Found 13800 images belonging to 46 classes.


    Defining the CNN's architecture:

In [5]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(46, activation='softmax')  
])

    Configuring the CNN:
    

In [6]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

    Fit the CNN:

In [7]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 128,  
    epochs=20,
    validation_data=test_generator,
    validation_steps=test_generator.samples // 128
)

Epoch 1/20
610/610 [==============================] - 53s 86ms/step - loss: 2.9047 - accuracy: 0.2283 - val_loss: 1.3518 - val_accuracy: 0.6094
Epoch 2/20
610/610 [==============================] - 55s 90ms/step - loss: 1.9935 - accuracy: 0.4435 - val_loss: 0.8768 - val_accuracy: 0.7434
Epoch 3/20
610/610 [==============================] - 55s 90ms/step - loss: 1.6100 - accuracy: 0.5419 - val_loss: 0.7952 - val_accuracy: 0.7569
Epoch 4/20
610/610 [==============================] - 54s 88ms/step - loss: 1.3999 - accuracy: 0.5988 - val_loss: 0.7688 - val_accuracy: 0.7646
Epoch 5/20
610/610 [==============================] - 54s 89ms/step - loss: 1.2495 - accuracy: 0.6386 - val_loss: 0.5884 - val_accuracy: 0.8215
Epoch 6/20
610/610 [==============================] - 54s 89ms/step - loss: 1.1419 - accuracy: 0.6658 - val_loss: 0.5189 - val_accuracy: 0.8402
Epoch 7/20
610/610 [==============================] - 54s 89ms/step - loss: 1.0750 - accuracy: 0.6853 - val_loss: 0.4879 - val_accuracy:

    Let's look at our model's performance:

In [9]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']